In [1]:
import pandas as pd
import traceback

import OpenDartReader
api_key = 'f2e08d4ed3de0ba3d5cbf59c04c223e02b1751a2'
dart = OpenDartReader(api_key)

In [2]:
def str2num(x):
    if "," in x:
        x = x.replace(",","")
    if x.startswith("(") and x.endswith(")"):
        x = x.lstrip("(")
        x = x.rstrip(")")
        float_num = float(x) * (-1)
    else:
        x = x.lstrip("(")
        x = x.rstrip(")")     
        float_num = float(x)
    return float_num

In [3]:
reports = dart.list('010060', start='2021-01-01', end='2021-12-31', kind='A', final=True)
# reports = dart.list('096640', start='2021-01-01', end='2021-12-31', kind='A', final=True)
# reports = dart.list('008290', start='2021-01-01', end='2021-12-31', kind='A', final=True)
# reports = dart.list('307750', start='2021-01-01', end='2021-12-31', kind='A', final=True)

In [4]:
reports

,corp_code,corp_name,stock_code,corp_cls,report_nm,rcept_no,flr_nm,rcept_dt,rm
0,00148896,OCI홀딩스,010060,Y,분기보고서 (2021.09),20211112000360,OCI홀딩스,20211112,
1,00148896,OCI홀딩스,010060,Y,반기보고서 (2021.06),20210813000353,OCI홀딩스,20210813,
2,00148896,OCI홀딩스,010060,Y,[기재정정]분기보고서 (2021.03),20210518000324,OCI홀딩스,20210518,
3,00148896,OCI홀딩스,010060,Y,[기재정정]사업보고서 (2020.12),20210401002874,OCI홀딩스,20210401,연


In [5]:
subreports = []
str_expr = "title.str.contains('5. 재무제표 주석')"
for i, row in reports.iterrows():
    if '사업보고서' in row['report_nm']:
        rpt_num = row['rcept_no']
        rpt_url = 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=' + rpt_num
        df_rpt = dart.sub_docs(rpt_num)
        rpt_data = df_rpt.query(str_expr)       # data frame
        rpt_title = rpt_data.iloc[0]['title']
        rpt_notes_url = rpt_data.iloc[0]['url']
        subreports.append({'rpt_num' : rpt_num, 'stock_code' : row['stock_code'], 'corp_name' : row['corp_name'], 'rpt_name' : row['report_nm'], 'rpt_url' : rpt_url,
                           'rpt_title' : rpt_title, 'rpt_notes_url' : rpt_notes_url}) 

In [6]:
subreports

[{'rpt_num': '20210401002874',
  'stock_code': '010060',
  'corp_name': 'OCI홀딩스',
  'rpt_name': '[기재정정]사업보고서 (2020.12)',
  'rpt_url': 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20210401002874',
  'rpt_title': '5. 재무제표 주석',
  'rpt_notes_url': 'http://dart.fss.or.kr/report/viewer.do?rcpNo=20210401002874&dcmNo=7952875&eleId=19&offset=1229304&length=492495&dtd=dart3.xsd'}]

In [7]:
data_list = []
for r in subreports:
    try:
        url = r['rpt_notes_url']
        doc_data = pd.read_html(url)
        val_a = val_b = None
        for df in doc_data:
            row = df.shape[0]
            for i in range(0, row):
                item = str(df.iloc[i,0])
                if item.__contains__('확정급여채무의 현재가치'):
                    # print(i, type(df.iloc[i,1]), df.iloc[i,1])
                    val_a = str2num(str(df.iloc[i,1]))
                elif item.__contains__('사외적립자산의 공정가치'):
                    # print(i, type(df.iloc[i,1]), df.iloc[i,1])                            
                    val_b = str2num(str(df.iloc[i,1]))
                else:
                    continue              
        data_list.append({'corp_name' : r['corp_name'], 'stock_code' : r['stock_code'], 'rpt_num' : r['rpt_num'], 'rpt_name' : r['rpt_name'], 
                        'val_a' : val_a, 'val_b' : val_b})
    except Exception as e:
        print(r['rpt_num'])
        print(r['rpt_url'])
        print(r['rpt_notes_url'])
        err_msg = traceback.format_exc()
        print(err_msg)

20210401002874
http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20210401002874
http://dart.fss.or.kr/report/viewer.do?rcpNo=20210401002874&dcmNo=7952875&eleId=19&offset=1229304&length=492495&dtd=dart3.xsd
Traceback (most recent call last):
  File "C:\Users\kyungho\AppData\Local\Temp\ipykernel_5736\1590050064.py", line 13, in <module>
    val_a = str2num(str(df.iloc[i,1]))
NameError: name 'str2num' is not defined



In [66]:
data_list

[{'corp_name': 'OCI홀딩스',
  'stock_code': '010060',
  'rpt_num': '20210401002874',
  'rpt_name': '[기재정정]사업보고서 (2020.12)',
  'val_a': 150685632.0,
  'val_b': -136848167.0}]